# Heuristic 평가

Heuristic 평가는 불충분한 시간이나 정보로 인해 완벽하게 합리적인 판단을 할 수 없을 때, 빠르고 간편하게 사용할 수 있는 추론 방법입니다.

(이는 LLM as Judge 를 활용할 때 드는 시간과 비용을 절약할 수 있다는 강점을 가지고 있기도 합니다.)



(참고) 아래의 코드 주석을 해제하여 라이브러리를 업데이트 후 진행합니다.

In [ ]:
# !pip install -qU langsmith langchain-teddynote rouge-score

In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [5]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

`ask_question` 이라는 이름으로 함수를 생성합니다. 입력으로는 `inputs` 라는 딕셔너리를 받고, 출력으로는 `answer` 라는 딕셔너리를 반환합니다.

In [6]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

## 한글 형태소 분석기의 활용

한글 형태소 분석기는 한국어 문장을 가장 작은 의미 단위인 형태소로 분리하고 각 형태소의 품사를 판별하는 도구입니다. 

형태소 분석기의 주요 기능
- 문장을 형태소 단위로 분리
- 각 형태소의 품사 태깅
- 형태소의 기본형 추출

Kiwipiepy 라이브러리를 활용하여 한글 형태소 분석기를 사용할 수 있습니다.

In [7]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

# 토크나이저 선언
kiwi_tokenizer = KiwiTokenizer()

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"

# 토큰화
print(sent1.split())
print(sent2.split())

print("===" * 20)

# 토큰화
print(kiwi_tokenizer.tokenize(sent1))
print(kiwi_tokenizer.tokenize(sent2))

['안녕하세요.', '반갑습니다.', '내', '이름은', '테디입니다.']
['안녕하세용', '반갑습니다~^^', '내', '이름은', '테디입니다!!']
['안녕', '하', '세요', '.', '반갑', '습니다', '.', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '.']
['안녕', '하', '세요', 'ᆼ', '반갑', '습니다', '~', '^^', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '!!']


## Rouge (Recall-Oriented Understudy for Gisting Evaluation) 스코어

- 자동 요약 및 기계 번역의 품질을 평가하는 데 사용되는 평가지표 입니다.
- 생성된 텍스트가 참조 텍스트의 중요 키워드를 얼마나 포함하는지 측정합니다.
- n-gram 중첩을 기반으로 계산됩니다

**Rouge-1**
- 단어 단위의 유사도를 측정합니다.
- 두 문장간의 개별 단어 일치도를 평가합니다.

**Rouge-2**
- 두 단어 연속(bigram)의 중복 단위의 유사도를 측정합니다.
- 두 문장간의 연속된 두 단어 일치도를 평가합니다.
  
**Rouge-L**
- 최장 공통 부분 수열(Longest Common Subsequence, LCS)을 기반으로 한 유사도를 측정합니다.
- 문장 수준의 단어 순서를 고려하며, 연속적인 일치를 요구하지 않습니다
- 더 유연한 평가가 가능하며, 문장 구조의 유사성을 자연스럽게 반영합니다.

In [8]:
from rouge_score import rouge_scorer

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], use_stemmer=False, tokenizer=KiwiTokenizer()
)

print(
    f"[1] {sent1}\n[2] {sent2}\n[rouge1] {scorer.score(sent1, sent2)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent2)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent2)['rougeL'].fmeasure:.5f}"
)
print("===" * 20)
print(
    f"[1] {sent1}\n[2] {sent3}\n[rouge1] {scorer.score(sent1, sent3)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent3)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent3)['rougeL'].fmeasure:.5f}"
)

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[rouge1] 0.77419
[rouge2] 0.62069
[rougeL] 0.77419
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[rouge1] 1.00000
[rouge2] 0.92857
[rougeL] 0.53333


## BLEU (Bilingual Evaluation Understudy) 스코어

주로 기계 번역 평가에 사용됩니다. 생성된 텍스트가 참조 텍스트와 얼마나 유사한지를 측정합니다.

n-gram 정밀도(precision)를 기반으로 계산됩니다

**계산 방식**
- N-gram 정밀도 계산: 기계 번역 결과에서 1-gram부터 4-gram까지의 n-gram이 참조 번역에 얼마나 포함되는지 계산합니다.
- 간결성 페널티(Brevity Penalty) 적용: 기계 번역이 참조 번역보다 짧을 경우 페널티를 부과합니다.
- 최종 점수 계산: N-gram 정밀도의 기하평균에 간결성 페널티를 곱하여 최종 BLEU 점수를 산출합니다

**한계점**
- 의미를 고려하지 않고 단순 문자열 일치만 확인합니다.
- 단어의 중요도를 구분하지 않습니다.

In [9]:
from nltk.translate.bleu_score import sentence_bleu

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# 토큰화
print(kiwi_tokenizer.tokenize(sent1, type="sentence"))
print(kiwi_tokenizer.tokenize(sent2, type="sentence"))
print(kiwi_tokenizer.tokenize(sent3, type="sentence"))

안녕 하 세요 . 반갑 습니다 . 나 의 이름 은 테디 이 ᆸ니다 .
안녕 하 세요 ᆼ 반갑 습니다 ~ ^^ 나 의 이름 은 테디 이 ᆸ니다 !!
나 의 이름 은 테디 이 ᆸ니다 . 안녕 하 세요 . 반갑 습니다 .


In [10]:
bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent2, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent2}\n[score] {bleu_score:.5f}")
print("===" * 20)

bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent3, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {bleu_score:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.74879
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.95739


## METEOR 스코어

기계 번역의 품질을 평가하기 위해 개발된 평가지표 입니다.

- BLEU의 단점을 보완하기 위해 개발되었습니다.
- 단순 단어 매칭 외에도 어간 추출(stemming), 동의어 매칭, 패러프레이징 등 다양한 언어학적 요소를 고려합니다.
- 단어 순서를 고려하여 평가합니다.
- 여러 참조 번역을 사용할 수 있습니다.
- 0에서 1 사이의 점수를 산출하며, 1에 가까울수록 좋은 번역을 의미합니다

In [11]:
from nltk.corpus import wordnet as wn

wn.ensure_loaded()

In [12]:
from nltk.translate import meteor_score

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent2, type="list"),
)

print(f"[1] {sent1}\n[2] {sent2}\n[score] {meteor:.5f}")
print("===" * 20)

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent3, type="list"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {meteor:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.78849
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.96800


## SemScore

- [SEMSCORE: Automated Evaluation of Instruction-Tuned LLMs based on Semantic Textual Similarity](https://arxiv.org/pdf/2401.17072)

이 논문에서는 의미론적 텍스트 유사성(STS)을 사용하여 모델 출력을 황금 표준 응답과 직접 비교하는 SEMSCORE라는 간단하지만 효과적인 평가 지표를 제안합니다. 12개의 주요 지시 튜닝된 LLM의 출력을 8개의 널리 사용되는 텍스트 생성 평가 지표로 비교 평가한 결과, 제안된 SEMSCORE 지표가 인간 평가와의 상관관계 측면에서 다른 모든 평가 지표보다 우수한 성능을 보여주었습니다.

`SentenceTransformer` 모델을 사용하여 문장 임베딩을 생성하고, 두 문장 간의 코사인 유사도를 계산합니다.
- 논문에서 사용된 모델인 `all-mpnet-base-v2` 를 사용합니다.

In [2]:
from sentence_transformers import SentenceTransformer, util
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# SentenceTransformer 모델 로드
model = SentenceTransformer("all-mpnet-base-v2")

# 문장들을 인코딩
sent1_encoded = model.encode(sent1, convert_to_tensor=True)
sent2_encoded = model.encode(sent2, convert_to_tensor=True)
sent3_encoded = model.encode(sent3, convert_to_tensor=True)

# sent1과 sent2 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent2_encoded).item()
print(f"[1] {sent1}\n[2] {sent2}\n[score] {cosine_similarity:.5f}")

print("===" * 20)

# sent1과 sent3 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent3_encoded).item()
print(f"[1] {sent1}\n[2] {sent3}\n[score] {cosine_similarity:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.86157
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.99191


위의 내용을 종합하여 정리한 Evaluator 는 다음과 같습니다.

In [13]:
from langsmith.schemas import Run, Example
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from sentence_transformers import SentenceTransformer, util
import os

# 토크나이저 병렬화 설정(HuggingFace 모델 사용)
os.environ["TOKENIZERS_PARALLELISM"] = "true"


def rouge_evaluator(metric: str = "rouge1") -> dict:
    # wrapper function 정의
    def _rouge_evaluator(run: Run, example: Example) -> dict:
        # 출력값과 정답 가져오기
        student_answer = run.outputs.get("answer", "")
        reference_answer = example.outputs.get("answer", "")

        # ROUGE 점수 계산
        scorer = rouge_scorer.RougeScorer(
            ["rouge1", "rouge2", "rougeL"], use_stemmer=True, tokenizer=KiwiTokenizer()
        )
        scores = scorer.score(reference_answer, student_answer)

        # ROUGE 점수 반환
        rouge = scores[metric].fmeasure

        return {"key": "ROUGE", "score": rouge}

    return _rouge_evaluator


def bleu_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="sentence")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="sentence")

    # BLEU 점수 계산
    bleu_score = sentence_bleu([reference_tokens], student_tokens)

    return {"key": "BLEU", "score": bleu_score}


def meteor_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="list")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="list")

    # METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    return {"key": "METEOR", "score": meteor}


def semscore_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # SentenceTransformer 모델 로드
    model = SentenceTransformer("all-mpnet-base-v2")

    # 문장 임베딩 생성
    student_embedding = model.encode(student_answer, convert_to_tensor=True)
    reference_embedding = model.encode(reference_answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cosine_similarity = util.pytorch_cos_sim(
        student_embedding, reference_embedding
    ).item()

    return {"key": "sem_score", "score": cosine_similarity}

Heuristic Evaluator 를 활용한 평가를 진행합니다.

In [14]:
from langsmith.evaluation import evaluate

# 평가자 정의
heuristic_evalulators = [
    rouge_evaluator(metric="rougeL"),
    bleu_evaluator,
    meteor_evaluator,
    semscore_evaluator,
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실험 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=heuristic_evalulators,
    experiment_prefix="Heuristic-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Heuristic-EVAL (Rouge, BLEU, METEOR, SemScore) 을 사용하여 평가",
    },
)

View the evaluation results for experiment: 'Heuristic-EVAL-07dd6e08' at:
https://smith.langchain.com/o/42ebd69b-2565-441c-b868-9709c2e20267/datasets/7be96c7d-41ca-4f3a-96da-7bf7c706e0a5/compare?selectedSessions=be515974-0bb3-44ad-bf85-290dc54c667a




0it [00:00, ?it/s]

결과를 확인합니다.

![](./assets/eval-07.png)